In [2]:
import pandas as pd
from tqdm import tqdm
from sklearn import preprocessing
from xgboost import XGBClassifier

In [4]:
train_data = pd.read_csv('../Database/train.csv')

In [5]:
train_data.head(3)

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,36 months,6 years,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,60 months,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,36 months,5 years,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A


대출기간 : 36 months , 60 months   </br>
주택소유상태 : MORTGAGE,RENT,own,ANY </br>
대출목적 : 부채 통합 ,신용 카드,주택 개선 ,기타 ,주요 구매  ,의료 ,자동차,소규모 사업 ,이사 ,휴가, 주택 ,재생 에너지          
     
       

 

In [6]:
label_encoder = preprocessing.LabelEncoder()
train_data['대출기간'] = label_encoder.fit_transform(train_data['대출기간'] )
train_data['주택소유상태'] = label_encoder.fit_transform(train_data['주택소유상태'] )
train_data['대출목적'] = label_encoder.fit_transform(train_data['대출목적'] )


In [7]:
train_data.head(3)

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,0,6 years,3,72000000,18.90,15,1,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,1,10+ years,1,130800000,22.33,21,10,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,0,5 years,1,96000000,8.60,14,1,0,928644,151944.0,0.0,0.0,A


근로기간

In [8]:
train_data['근로기간'].value_counts()

근로기간
10+ years    31585
2 years       8450
< 1 year      7774
3 years       7581
1 year        6249
Unknown       5671
5 years       5665
4 years       5588
8 years       4888
6 years       3874
7 years       3814
9 years       3744
10+years       896
<1 year        370
3               89
1 years         56
Name: count, dtype: int64

In [11]:
train_data['근로기간'] = train_data['근로기간'].str.extract(r'(\d+)')
train_data['근로기간']  =train_data['근로기간'].fillna(0)
train_data['근로기간']  =train_data['근로기간'].astype(int)

AttributeError: Can only use .str accessor with string values!

In [10]:
train_data['근로기간']

0         6
1        10
2         5
3         8
4         0
         ..
96289    10
96290    10
96291     1
96292     5
96293    10
Name: 근로기간, Length: 96294, dtype: int32

main processing

In [55]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [56]:
train_data = train_data.drop(columns='ID')
train_data_target = train_data['대출등급']
train_data = train_data.drop(columns='대출등급')

In [ ]:
train_data_x, valid_data_x, train_data_y, valid_data_y= train_test_split(train_data,train_data_target,test_size=0.2, shuffle=True,random_state=34)

## **모델적용**

In [ ]:
import optuna
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np

# 데이터 준비 (X_train, y_train 등은 사전에 준비되어야 합니다)
# 예: X_train, X_test, y_train, y_test = train_test_split(...)

def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 10000),
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.1, 5),
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1),
        'subsample': trial.suggest_float('subsample', 0.5, 1)
    }

    model = XGBClassifier(**param, random_state=1004)
    model.fit(train_data_x, train_data_y)
    preds = model.predict(valid_data_x)
    accuracy = accuracy_score(valid_data_y, preds)
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)  # 예를 들어 100번의 시도

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)


In [44]:

model = XGBClassifier(random_state=1004, n_estimators=8228, max_depth=12, min_child_weight=1, gamma=3, learning_rate=0.0001,
                      colsample_bytree=1.0, reg_lambda=0.767800554, reg_alpha=0.01232, subsample=0.6)

## **valid set도 train에 포함**

In [24]:
model = RandomForestClassifier(n_estimators=2000,max_depth=40, criterion='entropy', random_state=42, n_jobs=-1)
model.fit(train_data, train_data_target)

RandomForestClassifier(criterion='entropy', max_depth=40, n_estimators=2000,
                       n_jobs=-1, random_state=42)

## **test**

In [25]:
test_data = pd.read_csv('./test.csv')
test_data.head(2)

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
0,TEST_00000,16800000,36 months,8 years,MORTGAGE,132000000,19.64,12,주택 개선,0,394692,146604.0,0.0,0.0
1,TEST_00001,8400000,36 months,5 years,RENT,89971200,15.84,25,부채 통합,0,0,0.0,0.0,0.0


In [26]:
test_data['대출기간'] = label_encoder.fit_transform(test_data['대출기간'] )
test_data['주택소유상태'] = label_encoder.fit_transform(test_data['주택소유상태'] )
test_data['대출목적'] = label_encoder.fit_transform(test_data['대출목적'] )
test_data['근로기간'] = test_data['근로기간'].str.extract(r'(\d+)')
test_data['근로기간']  =test_data['근로기간'].fillna(0)
test_data['근로기간']  =test_data['근로기간'].astype(int)

In [27]:
test_data = test_data.drop(columns='ID')

In [28]:
test_pred = model.predict(test_data)

submissionfile 생성

In [29]:
submission_data = pd.read_csv('./sample_submission.csv')
submission_data['대출등급'] = test_pred
submission_data.to_csv('./randomforest.csv', index=False)


In [30]:
submission_data

,ID,대출등급
0,TEST_00000,B
1,TEST_00001,B
2,TEST_00002,A
3,TEST_00003,C
4,TEST_00004,C
...,...,...
64192,TEST_64192,D
64193,TEST_64193,D
64194,TEST_64194,C
64195,TEST_64195,C
